In [2]:
from sqlalchemy import create_engine, Column, Integer, String, Date, Numeric
from sqlalchemy.orm import declarative_base, sessionmaker
from datetime import date

# -----------------------------------------
# 1. Setup SQLite database (in-memory or file)
# -----------------------------------------
# Use ':memory:' for temporary DB, or 'sqlite:///metalprices.db' to persist
engine = create_engine('sqlite:///metalprices.db', echo=False)
Base = declarative_base()

# -----------------------------------------
# 2. Define Table Schema
# -----------------------------------------
class MetalPrice(Base):
    __tablename__ = 'metalprices'

    Id = Column(Integer, primary_key=True, autoincrement=True)
    Date = Column(Date, nullable=False)
    Metal = Column(String(50), nullable=False)
    Price = Column(Numeric(18, 4), nullable=False)

    def __repr__(self):
        return f"<MetalPrice(Id={self.Id}, Date={self.Date}, Metal='{self.Metal}', Price={self.Price})>"

# Create the table
Base.metadata.create_all(engine)

# -----------------------------------------
# 3. Create a session
# -----------------------------------------
Session = sessionmaker(bind=engine)
session = Session()

# -----------------------------------------
# 4. CREATE: Insert new entries
# -----------------------------------------
print("Inserting records...\n")
session.add_all([
    MetalPrice(Date=date(2021, 1, 1), Metal='Copper', Price=9000.50),
    MetalPrice(Date=date(2021, 1, 1), Metal='Zinc', Price=3000.75),
    MetalPrice(Date=date(2021, 2, 1), Metal='Copper', Price=9200.00),
    MetalPrice(Date=date(2021, 2, 1), Metal='Zinc', Price=3100.00),
    MetalPrice(Date=date(2021, 3, 1), Metal='Zinc', Price=3200.00),
])
session.commit()

# -----------------------------------------
# 5. READ: Retrieve data from the table
# -----------------------------------------
print("All records:\n")
for record in session.query(MetalPrice).all():
    print(record)

print("\nAll Zinc records:\n")
zinc_prices = session.query(MetalPrice).filter_by(Metal='Zinc').all()
for record in zinc_prices:
    print(record)

print("\nCopper prices above 9000:\n")
high_copper = session.query(MetalPrice).filter(
    MetalPrice.Metal == 'Copper',
    MetalPrice.Price > 9000
).all()
for record in high_copper:
    print(record)

# -----------------------------------------
# 6. UPDATE: Modify a price
# -----------------------------------------
print("\nUpdating Copper price on 2021-01-01...\n")
copper_jan = session.query(MetalPrice).filter_by(Metal='Copper', Date=date(2021, 1, 1)).first()
if copper_jan:
    copper_jan.Price = 9050.00
    session.commit()
    print("Updated:", copper_jan)

# -----------------------------------------
# 7. DELETE: Remove a specific entry
# -----------------------------------------
print("\nDeleting Zinc price on 2021-03-01...\n")
zinc_mar = session.query(MetalPrice).filter_by(Metal='Zinc', Date=date(2021, 3, 1)).first()
if zinc_mar:
    session.delete(zinc_mar)
    session.commit()
    print(f"Deleted record for Zinc on 2021-03-01")

# -----------------------------------------
# 8. Additional CRUD: Count & Existence check
# -----------------------------------------
total_records = session.query(MetalPrice).count()
print(f"\nTotal records in table: {total_records}")

exists = session.query(MetalPrice).filter_by(Metal='Zinc', Date=date(2021, 3, 1)).first()
if exists:
    print("Zinc price on 2021-03-01 still exists.")
else:
    print("Zinc price on 2021-03-01 has been successfully deleted.")

# -----------------------------------------
# 9. Final read to confirm current state
# -----------------------------------------
print("\nFinal state of table:\n")
for record in session.query(MetalPrice).order_by(MetalPrice.Date).all():
    print(record)



Inserting records...

All records:

<MetalPrice(Id=1, Date=2021-01-01, Metal='Copper', Price=9000.5000)>
<MetalPrice(Id=2, Date=2021-01-01, Metal='Zinc', Price=3000.7500)>
<MetalPrice(Id=3, Date=2021-02-01, Metal='Copper', Price=9200.0000)>
<MetalPrice(Id=4, Date=2021-02-01, Metal='Zinc', Price=3100.0000)>
<MetalPrice(Id=5, Date=2021-03-01, Metal='Zinc', Price=3200.0000)>

All Zinc records:

<MetalPrice(Id=2, Date=2021-01-01, Metal='Zinc', Price=3000.7500)>
<MetalPrice(Id=4, Date=2021-02-01, Metal='Zinc', Price=3100.0000)>
<MetalPrice(Id=5, Date=2021-03-01, Metal='Zinc', Price=3200.0000)>

Copper prices above 9000:

<MetalPrice(Id=1, Date=2021-01-01, Metal='Copper', Price=9000.5000)>
<MetalPrice(Id=3, Date=2021-02-01, Metal='Copper', Price=9200.0000)>

Updating Copper price on 2021-01-01...

Updated: <MetalPrice(Id=1, Date=2021-01-01, Metal='Copper', Price=9050.0000)>

Deleting Zinc price on 2021-03-01...

Deleted record for Zinc on 2021-03-01

Total records in table: 4
Zinc price on 2